In [2]:
# TODO : CTC Loss
from typing import List

import cv2
import gdown
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import Compose, Lambda

#import tensorflow as tf

In [3]:
import os
import cv2
import dlib
import numpy as np
from tqdm import tqdm

In [6]:
import cv2
import dlib
import pandas as pd
import os
from tqdm import tqdm
# Define paths
data_path = 'data3/s3'

# Initialize dlib's face detector (HOG-based) and then create the facial landmark predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("data_kaggle/shape_predictor_68_face_landmarks.dat")

# Function to extract keypoints
def extract_keypoints_from_frame(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    rects = detector(gray, 1)
    keypoints = []
    for rect in rects:
        shape = predictor(gray, rect)
        for i in range(0, 68):
            keypoints.append((shape.part(i).x, shape.part(i).y))
    return keypoints

# List to store results
results = []

# Iterate through videos
for root, dirs, files in os.walk(data_path):
    for file in tqdm(files, total=1000):
        if file.endswith(".mpg"):  # Add other video formats if needed
            video_path = os.path.join(root, file)
            cap = cv2.VideoCapture(video_path)
            frame_number = 0
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break
                keypoints = extract_keypoints_from_frame(frame)
                for idx, (x, y) in enumerate(keypoints):
                    results.append({'video_path': video_path, 'frame_number': frame_number, 'x': x, 'y': y, 'point_index': idx})
                frame_number += 1
            cap.release()

# Convert list to DataFrame
df = pd.DataFrame(results)

# Save DataFrame to CSV
df.to_csv('facial_keypoints.csv', index=False)

print("Keypoints extraction and saving to CSV completed.")


1001it [46:08,  2.77s/it]                          


Keypoints extraction and saving to CSV completed.


In [53]:
df = pd.read_csv("facial_keypoints.csv")
video_path_counts = df.video_path.value_counts()
video_paths_to_keep = video_path_counts[video_path_counts == 5100].index
df = df[df['video_path'].isin(video_paths_to_keep)]

df_filtered = df.copy()

In [56]:
chunks = []

num_frames_per_chunk = 5100
frames_per_video = 75
keypoints_per_frame = 68
coords_per_keypoint = 2

for video_path, group in df_filtered.groupby('video_path'):
    num_frames = len(group) // keypoints_per_frame
    num_chunks = num_frames // frames_per_video
    for i in range(num_chunks):
        chunk = group.iloc[i * frames_per_video * keypoints_per_frame:(i + 1) * frames_per_video * keypoints_per_frame]
        chunk_reshaped = chunk[['x', 'y']].values.reshape(-1, keypoints_per_frame, coords_per_keypoint)
        chunks.append(chunk_reshaped)

chunks_array = np.array(chunks)
print(f"Data shape: {chunks_array.shape}")

Data shape: (986, 75, 68, 2)


In [57]:
chunks_array.shape

(986, 75, 68, 2)

In [58]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split

In [59]:
def create_vocab():
    vocab = "abcdefghijklmnopqrstuvwxyz123456789 "
    return vocab


def char_to_int(char):
    # shift 1 
    vocab = "abcdefghijklmnopqrstuvwxyz123456789 "
    return vocab.index(char) + 1 if char in vocab else -1

def int_to_char(index):
    # shift 1 
    vocab = "abcdefghijklmnopqrstuvwxyz123456789 "
    return vocab[index - 1] if 1 <= index <= len(vocab) else ''

def load_alignments(path:str) -> List[str]:
    with open(path, 'r') as f:
        lines = f.readlines()
    tokens = []
    for line in lines:
        line = line.split()
        if line[2] != 'sil':
            tokens.extend([*line[2]])
            tokens.append(' ')
    return [char_to_int(token) for token in tokens]

all_alignments = []
for video_path in df['video_path'].unique():
    datapath = video_path.split('/')[0]
    speaker_path = video_path.split('/')[-1].split('\\')[0]
    vid_path = video_path.split('/')[-1].split('\\')[-1].split('.')[0]
    
    alignment_path = os.path.join(f'{datapath}','align',f'{vid_path}.align')
    alignments = load_alignments(alignment_path) 
    all_alignments.append(alignments)


In [60]:
max_len = max(len(label) for label in all_alignments)
label_data = [np.pad(label, (0, max_len - len(label)), 'constant', constant_values=0) for label in all_alignments]

# Convert label_data to numpy array
label_data = np.array(label_data)

In [61]:
label_data

array([[ 2,  9, 14, ...,  0,  0,  0],
       [ 2,  9, 14, ...,  0,  0,  0],
       [ 2,  9, 14, ...,  0,  0,  0],
       ...,
       [19,  5, 20, ...,  0,  0,  0],
       [19,  5, 20, ...,  0,  0,  0],
       [19,  5, 20, ...,  0,  0,  0]])

In [63]:
input_data_tensor = torch.tensor(chunks_array, dtype=torch.float32)
input_data_tensor.shape

torch.Size([986, 75, 68, 2])

In [65]:
label_data_tensor = torch.tensor(label_data, dtype=torch.long)
label_data_tensor.shape

torch.Size([986, 36])

In [66]:
dataset = TensorDataset(input_data_tensor, label_data_tensor)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [67]:
vocab = create_vocab()


In [86]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

class LipReadingRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(LipReadingRNN, self).__init__()
        self.hidden_size = hidden_size
        
        self.input_layer = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.LayerNorm(hidden_size),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.LayerNorm(hidden_size)
        )
        self.bilstm = nn.LSTM(hidden_size, hidden_size, num_layers=4, batch_first=True, bidirectional=True, dropout=0.3)
        self.gru = nn.GRU(hidden_size * 2, hidden_size, num_layers=4, batch_first=True, dropout=0.3)
        self.dropout = nn.Dropout(0.3)
        
        self.output_layer = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.LayerNorm(hidden_size),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, num_classes)
        )
        
        self._initialize_weights()
    
    def _initialize_weights(self):
        for layer in self.input_layer:
            if isinstance(layer, nn.Linear):
                nn.init.xavier_uniform_(layer.weight)
                nn.init.zeros_(layer.bias)
        
        for name, param in self.bilstm.named_parameters():
            if 'weight' in name:
                nn.init.xavier_uniform_(param)
            elif 'bias' in name:
                nn.init.zeros_(param)
                
        for layer in self.output_layer:
            if isinstance(layer, nn.Linear):
                nn.init.xavier_uniform_(layer.weight)
                nn.init.zeros_(layer.bias)

    def forward(self, x):
        x = self.input_layer(x)
        
        h0 = torch.zeros(8, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(8, x.size(0), self.hidden_size).to(x.device)
        
        out, _ = self.bilstm(x, (h0, c0))
        out = self.gru(out)[0]
        out = self.dropout(out)
        out = self.output_layer(out)
        
        return out

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

input_size = 68 * 2
hidden_size = 512
num_classes = len(vocab) + 1
model = LipReadingRNN(input_size, hidden_size, num_classes).to(device)
criterion = nn.CTCLoss(blank=0, reduction='mean', zero_infinity=True)
optimizer = optim.AdamW(model.parameters(), lr=1e-4)
scheduler = StepLR(optimizer, step_size=200, gamma=0.1)


print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.cuda.get_device_name(torch.cuda.current_device()))


num_epochs = 25
model.train()
for epoch in range(num_epochs):
    total_loss = 0
    model.train()
    for sequences, labels in train_loader:
        sequences = sequences.to(device)
        labels = labels.to(device)
        sequences = sequences.view(sequences.size(0), sequences.size(1), -1).to(device) # (batch_size, seq_len, input_size)
        
        outputs = model(sequences)
        
        outputs = outputs.permute(1, 0, 2)
        input_lengths = torch.full((sequences.size(0),), sequences.size(1), dtype=torch.long).to(device)
        target_lengths = torch.tensor([label[label != 0].size(0) for label in labels], dtype=torch.long).to(device)
        loss = criterion(outputs.log_softmax(2), labels, input_lengths, target_lengths).to(device)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()

    scheduler.step()

    model.eval()
    val_loss = 0
    with torch.no_grad():
        for sequences, labels in test_loader:
            sequences = sequences.to(device)
            labels = labels.to(device)
            sequences = sequences.view(sequences.size(0), sequences.size(1), -1).to(device) # (batch_size, seq_len, input_size)

            outputs = model(sequences)

            outputs = outputs.permute(1, 0, 2)
            input_lengths = torch.full((sequences.size(0),), sequences.size(1), dtype=torch.long).to(device)
            target_lengths = torch.tensor([label[label != 0].size(0) for label in labels], dtype=torch.long).to(device)
            loss = criterion(outputs.log_softmax(2), labels, input_lengths, target_lengths).to(device)

            val_loss += loss.item()
    print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {total_loss/len(train_loader):.4f}, Validation Loss: {val_loss/len(test_loader):.4f}')


True
0
NVIDIA GeForce RTX 3060
Epoch [1/25], Training Loss: 3.9520, Validation Loss: 3.3297
Epoch [2/25], Training Loss: 3.1447, Validation Loss: 3.0478
Epoch [3/25], Training Loss: 2.9758, Validation Loss: 2.8821
Epoch [4/25], Training Loss: 2.8342, Validation Loss: 2.7030
Epoch [5/25], Training Loss: 2.6812, Validation Loss: 2.6743
Epoch [6/25], Training Loss: 2.6079, Validation Loss: 2.5546
Epoch [7/25], Training Loss: 2.5213, Validation Loss: 2.4604
Epoch [8/25], Training Loss: 2.4378, Validation Loss: 2.3652
Epoch [9/25], Training Loss: 2.3349, Validation Loss: 2.2781
Epoch [10/25], Training Loss: 2.2343, Validation Loss: 2.1683
Epoch [11/25], Training Loss: 2.1591, Validation Loss: 2.0926
Epoch [12/25], Training Loss: 2.0912, Validation Loss: 2.0195
Epoch [13/25], Training Loss: 2.0261, Validation Loss: 1.9711
Epoch [14/25], Training Loss: 1.9563, Validation Loss: 1.8845
Epoch [15/25], Training Loss: 1.8975, Validation Loss: 1.8478
Epoch [16/25], Training Loss: 1.8485, Validation

In [87]:
import jiwer

def calculate_wer(reference, hypothesis):
    """
    Calculate the Word Error Rate (WER).
    - reference: The ground truth string.
    - hypothesis: The predicted string.
    Returns the WER as a float.
    """
    return jiwer.wer(reference, hypothesis)

def calculate_cer(reference, hypothesis):
    """
    Calculate the Character Error Rate (CER).
    - reference: The ground truth string.
    - hypothesis: The predicted string.
    Returns the CER as a float.
    """
    return jiwer.cer(reference, hypothesis)

def ctc_greedy_decoder(output, int_to_char, blank_label):
    """
    Decodes the output of the network using a greedy approach.
    - output: The raw output from the network.
    - int2char: A dictionary mapping indices to characters.
    - blank_label: The index of the blank label.
    Returns a list of decoded words.
    """
    decoded_words = []
    for batch in output:
        word = []
        prev_char = None
        for i in batch:
            char_idx = i.item()
            if char_idx != blank_label and (prev_char is None or char_idx != prev_char):
                word.append(int_to_char(char_idx))
            prev_char = char_idx
        decoded_words.append(''.join(word))
    return decoded_words

# Ensure the blank label is mapped to a space character if needed
# int2char[blank_label] = ' '
model.eval()
with torch.no_grad():
    total_val_loss = 0
    all_golden_words = []
    all_predicted_words = []
    for sequences, labels in test_loader:
        sequences = sequences.view(sequences.size(0), sequences.size(1), -1)  # Flatten the input dimensions
        
        # Print the shape of sequences
        print(f'Validation Batch, sequences shape: {sequences.shape}')
        
        # Move tensors to the appropriate device
        sequences = sequences.to(device)
        labels = labels.to(device)
        
        outputs = model(sequences)
        
        # Convert outputs to predicted indices
        max_indices = torch.argmax(outputs, dim=2)
        
        # Decode the predicted indices to words
        predicted_words = ctc_greedy_decoder(max_indices, int_to_char, 0)
        
        # Convert golden labels to characters
        golden_words = []
        for label in labels:
            word = []
            for i in label:
                if i.item() != 0:  # Ignore the padding (blank) label
                    word.append(int_to_char(i.item()))
            golden_words.append(''.join(word))
        
        # Collect all golden and predicted words
        all_golden_words.extend(golden_words)
        all_predicted_words.extend(predicted_words)
        
        # Print the golden and predicted labels
        for i in range(len(golden_words)):
            print(f"Golden: {golden_words[i]}")
            print(f"Predicted: {predicted_words[i]}")
            
        # Only process the first batch for printing
        break


Validation Batch, sequences shape: torch.Size([32, 75, 136])
Golden: place white by x seven soon 
Predicted: sla bree at ie no
Golden: bin green by b zero please 
Predicted: sla bree at ie no
Golden: set green by p five sp again 
Predicted: sla bree at ie no
Golden: set white with v seven again 
Predicted: sla bree at ie no
Golden: bin blue by z eight please 
Predicted: sla bree at ie no
Golden: lay green in r six now 
Predicted: sla bree at ie no
Golden: place red by p seven soon 
Predicted: sla bree at ie no
Golden: set red with sp b nine again 
Predicted: sla bree at ie no
Golden: place blue at v three again 
Predicted: sla bree at ie no
Golden: lay green with t zero please 
Predicted: sla bree at ie no
Golden: set white with v four now 
Predicted: sla bree at ie no
Golden: place white by j nine soon 
Predicted: sla bree at ie no
Golden: set white sp at b five soon 
Predicted: sla bree at ie no
Golden: set white in n nine sp soon 
Predicted: sla bree at ie no
Golden: bin green with 